In [ ]:
import os
import zipfile
import glob
import shutil
import nibabel as nib
import numpy as np

def extract_zip(zip_file_path, extract_dir):
    """Extracts the contents of the zip file into the specified directory."""
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Extracted zip file to {extract_dir}")

def threshold_nii_file(file_path, threshold_value, save_dir):
    """Loads a .nii file, applies a threshold, and saves the result."""
    # Load the .nii file
    nii_img = nib.load(file_path)
    nii_data = nii_img.get_fdata()
    nii_aff = nii_img.affine
    nii_hdr = nii_img.header

    # Apply threshold
    thresholded_data = np.where(nii_data > threshold_value, nii_data, 0)

    # Save the thresholded data to a new .nii file
    filename = os.path.basename(file_path)
    save_path = os.path.join(save_dir, filename)
    thresholded_img = nib.Nifti1Image(thresholded_data, nii_aff, nii_hdr)
    nib.save(thresholded_img, save_path)
    print(f"Processed and saved: {save_path}")

def main(zip_file_path, extract_dir, save_dir, threshold_value):
    # Step 1: Extract zip file
    extract_zip(zip_file_path, extract_dir)

    # Step 2: Process each .nii file one by one using glob
    nii_files = glob.glob(os.path.join(extract_dir+'/ribfrac-val-images/', '*.nii.gz'))
    os.makedirs(save_dir, exist_ok=True)
    
    for nii_file in nii_files:
        print(f"Processing file: {nii_file}")
        threshold_nii_file(nii_file, threshold_value, save_dir)
        os.remove(nii_file)  # Remove the file after processing to free up space

    # Clean up the extraction directory
    shutil.rmtree(extract_dir)
    print(f"Processing complete. Results saved in: {save_dir}")

if __name__ == "__main__":
    zip_file_path = '/workspace/RibCage/ribfrac-val-images.zip'
    extract_dir = './temp'
    save_dir = '/workspace/RibCage/val-segmented_ribfrac'
    threshold_value = 150

    main(zip_file_path, extract_dir, save_dir, threshold_value)

Extracted zip file to ./temp
Processing file: ./temp/ribfrac-val-images/RibFrac494-image.nii.gz
Processed and saved: /workspace/RibCage/val-segmented_ribfrac/RibFrac494-image.nii.gz
Processing file: ./temp/ribfrac-val-images/RibFrac437-image.nii.gz
Processed and saved: /workspace/RibCage/val-segmented_ribfrac/RibFrac437-image.nii.gz
Processing file: ./temp/ribfrac-val-images/RibFrac474-image.nii.gz
Processed and saved: /workspace/RibCage/val-segmented_ribfrac/RibFrac474-image.nii.gz
Processing file: ./temp/ribfrac-val-images/RibFrac467-image.nii.gz
Processed and saved: /workspace/RibCage/val-segmented_ribfrac/RibFrac467-image.nii.gz
Processing file: ./temp/ribfrac-val-images/RibFrac496-image.nii.gz
Processed and saved: /workspace/RibCage/val-segmented_ribfrac/RibFrac496-image.nii.gz
Processing file: ./temp/ribfrac-val-images/RibFrac477-image.nii.gz
Processed and saved: /workspace/RibCage/val-segmented_ribfrac/RibFrac477-image.nii.gz
Processing file: ./temp/ribfrac-val-images/RibFrac463

Processed and saved: /workspace/RibCage/val-segmented_ribfrac/RibFrac449-image.nii.gz
Processing file: ./temp/ribfrac-val-images/RibFrac456-image.nii.gz
Processed and saved: /workspace/RibCage/val-segmented_ribfrac/RibFrac456-image.nii.gz
Processing file: ./temp/ribfrac-val-images/RibFrac423-image.nii.gz
Processed and saved: /workspace/RibCage/val-segmented_ribfrac/RibFrac423-image.nii.gz
Processing file: ./temp/ribfrac-val-images/RibFrac488-image.nii.gz
Processed and saved: /workspace/RibCage/val-segmented_ribfrac/RibFrac488-image.nii.gz
Processing file: ./temp/ribfrac-val-images/RibFrac459-image.nii.gz
Processed and saved: /workspace/RibCage/val-segmented_ribfrac/RibFrac459-image.nii.gz
Processing file: ./temp/ribfrac-val-images/RibFrac439-image.nii.gz
Processed and saved: /workspace/RibCage/val-segmented_ribfrac/RibFrac439-image.nii.gz
Processing file: ./temp/ribfrac-val-images/RibFrac500-image.nii.gz
Processed and saved: /workspace/RibCage/val-segmented_ribfrac/RibFrac500-image.nii

In [1]:
import os
import zipfile
import glob
import shutil
import nibabel as nib
import numpy as np

class Window:
    def __init__(self, window_min, window_max):
        self.window_min = window_min
        self.window_max = window_max

    def __call__(self, image):
        # Clip image pixel values to within the specified window
        image = np.clip(image, self.window_min, self.window_max)
        return image

def extract_zip(zip_file_path, extract_dir):
    """Extracts the contents of the zip file into the specified directory."""
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Extracted zip file to {extract_dir}")

def threshold_nii_file(file_path, window, save_dir):
    """Loads a .nii file, applies window thresholding, and saves the result."""
    # Load the .nii file
    nii_img = nib.load(file_path)
    nii_data = nii_img.get_fdata()
    nii_aff = nii_img.affine
    nii_hdr = nii_img.header

    # Apply window thresholding
    thresholded_data = window(nii_data)

    # Save the thresholded data to a new .nii file
    filename = os.path.basename(file_path)
    save_path = os.path.join(save_dir, filename)
    thresholded_img = nib.Nifti1Image(thresholded_data, nii_aff, nii_hdr)
    nib.save(thresholded_img, save_path)
    print(f"Processed and saved: {save_path}")

def main(zip_file_path, extract_dir, save_dir, window_min, window_max):
    # Step 1: Extract zip file
    extract_zip(zip_file_path, extract_dir)

    # Step 2: Process each .nii file one by one using glob
    nii_files = glob.glob(os.path.join(extract_dir+'/ribfrac-val-images/', '*.nii.gz'))
    os.makedirs(save_dir, exist_ok=True)

    # Create a Window object with the specified thresholds
    window = Window(window_min, window_max)
    
    for nii_file in nii_files:
        print(f"Processing file: {nii_file}")
        threshold_nii_file(nii_file, window, save_dir)
        os.remove(nii_file)  # Remove the file after processing to free up space
        break

    # Clean up the extraction directory
    shutil.rmtree(extract_dir)
    print(f"Processing complete. Results saved in: {save_dir}")

if __name__ == "__main__":
    zip_file_path = '/workspace/RibCage/ribfrac-val-images.zip'
    extract_dir = './temp'
    save_dir = '/workspace/RibCage/val-segmented_ribfrac'
    
    # Window thresholds
    window_min = -1000
    window_max = 700

    main(zip_file_path, extract_dir, save_dir, window_min, window_max)

FileNotFoundError: [Errno 2] No such file or directory: '/workspace/RibCage/ribfrac-val-images.zip'